# Data Inconsistencies

In [1]:
import pandas as pd
from siuba import *

import numpy as np

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Markdown
from IPython.core.display import display

In [2]:
import _clean_data
import _dla_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabl

In [3]:
#df = _clean_data.make_clean_data()
df= pd.read_parquet("dla_df.parquet")

In [4]:
df.sample()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
7204,Obligated,HSIPL,5403(022),Carson,2016-02-23,2016-03-04,2016-03-08,2016-03-09,2016-03-16,418880.0,...,SCAG015,Intersection Of Avalon Blvd And Carson St. In ...,"Construct Two Right Turn Lanes,modigy Traffic ...",4,NaT,NaT,SCAG,5403,2016.0,Carson


**NOTE** this notebook was made before offical `primary_agency_name` and `locode` was known. `agency` is used to show the inconsistencies that arise when looking at certain `project_ids` and `locode`

## Looking into MPOs

using this map to see where the agency is in realtion to nearest mpo https://gisdata-caltrans.opendata.arcgis.com/datasets/b3e0ef95520843ba8c1d3b9c0fa9a607_0/explore

In [6]:
print(len(df>>count(_.mpo)>>arrange(-_.n)))

31


* Accoring to the Caltrans GIS data, there are 18 mpos

In [73]:
#using _dla_utils
(_dla_utils.find_top(df))>>filter(_.variable=='mpo')

,value,count,variable
0,SCAG,4988,mpo
1,MTC,3236,mpo
2,NON-MPO,2592,mpo
3,SACOG,1865,mpo
4,CFCG,1404,mpo
5,KCOG,998,mpo
6,STANCOG,759,mpo
7,AMBAG,745,mpo
8,SJCG,515,mpo
9,TCAG,489,mpo


### Which MPOs have the least amount of obligations? What Agencies do they include?

In [11]:
df >> count(_.mpo)>>arrange(-_.n)>> filter(_.n==1)

,mpo,n
6,FSTIP,1
11,MNOLTC,1
15,SANDA,1
20,SHASTA,1


#### MPO SANDA

In [12]:
df>>filter(_.mpo =='SANDA')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
12809,Obligated,HSIPLN,5426(008),Big Bear Lake,2013-12-09,2014-01-27,2014-02-06,2014-02-06,2014-02-07,169650.0,...,SCAG015,Big Bear Lake Boulevard (sr 18) From Paine Roa...,Widen Shoulder For Two Way Left Turn Lane And ...,1,NaT,NaT,SANDA,5426,2013.0,Big Bear Lake


* maybe meant "SANDAG" but Big Bear Lake is in "SCAG" jurisdiction
* looked to excel download file and this there is anothe Big Bear Lake and SANDAG instance

In [13]:
df>>filter(_.agency.str.contains('Big Bear')) >> filter(_.mpo.str.contains("SANDA"))

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
12609,Obligated,HSIPLN,5426(011),Big Bear Lake,2014-02-20,2014-02-21,2014-03-02,2014-03-03,2014-03-04,27000.0,...,SCAG015,"Big Bear Boulevard (sr 18) From 1,000 Ft. West...",Widen Shoulder For Two Way Left Turn Lane,1,NaT,NaT,SANDAG,5426,2014.0,Big Bear Lake
12809,Obligated,HSIPLN,5426(008),Big Bear Lake,2013-12-09,2014-01-27,2014-02-06,2014-02-06,2014-02-07,169650.0,...,SCAG015,Big Bear Lake Boulevard (sr 18) From Paine Roa...,Widen Shoulder For Two Way Left Turn Lane And ...,1,NaT,NaT,SANDA,5426,2013.0,Big Bear Lake


google maps show the `project_location` intersections in the jurisdiction of Big Bear and not in SANDAG

#### MPO SHASTA

In [14]:
df >>filter(_.mpo=='SHASTA')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
10079,Obligated,HSIPL,5073(075),Orange,2014-12-30,2015-01-08,2015-01-16,2015-01-16,2015-01-18,196199.0,...,SCAG015,Intersection Of Tustin Street And Mayfair Avenue,Left-turn Signal Phasing,2,NaT,NaT,SHASTA,5073,2014.0,Orange


In [15]:
df>> filter(_.agency.str.contains('Shasta')) >> count(_.mpo)

,mpo,n
0,MTC,1
1,SACOG,3
2,STPA,129


In [16]:
df>>filter(_.agency.str.contains('Shasta')) >> filter(_.mpo=="MTC")

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
19491,Obligated,ACSTP,15A6(002),Shasta County,2021-05-18,2021-05-28,2021-06-02,2021-07-12,2021-07-22,0.0,...,nan,Ca20-3 Zogg Fire Eo Repairs On Platina Road,Emergency Opening,1,2021-04-15,2021-04-20,MTC,15A6,2021.0,Napa County


* in the project location column, the intersection is Tustin and Mayfair which is in Orange Couny, or SCAG. Agency is also Orange, yet the mpo is Shasta

#### MPO MNOLTC and Agencies under

In [17]:
df>> filter(_.mpo=='MNOLTC')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
10862,Obligated,RPSTPL,5205(019),Rialto,2014-09-10,2014-09-10,2014-09-10,2014-09-10,2014-09-10,2000000.0,...,20040830,Former Pacific Electric Railroad Right-of-way ...,Extend Class 1 Bicycle Trail,2,NaT,NaT,MNOLTC,5205,2014.0,Rialto


In [18]:
#looking to see if what other MPOs Rialto is in, should be in SCAG
df>> filter(_.agency=='Rialto') >> count(_.mpo)

,mpo,n
0,KCOG,1
1,MNOLTC,1
2,SANDAG,1
3,SCAG,15


In [19]:
df>> filter(_.agency=='Rialto') >> filter(_.mpo=='KCOG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
10709,Obligated,SRTSL,5205(014),Rialto,2014-10-08,2014-10-08,2014-10-08,2014-10-08,2014-10-14,0.0,...,SCAG016,"Eucalyptus Avenue, King Street, 2nd Street, En...","Curb, Ramps, And In-pavement Warning Signals",3,NaT,NaT,KCOG,5205,2014.0,Rialto


In [20]:
df>> filter(_.agency=='Rialto') >> filter(_.mpo=='SANDAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
12458,Obligated,HSIPL,5205(017),Rialto,2014-03-07,2014-03-07,2014-03-21,2014-03-21,2014-03-21,-89313.0,...,SCAG015,Various Intersections Throughout The City Of R...,Pedestrian Countdown Heads At 67 Intersections,2,NaT,NaT,SANDAG,5205,2014.0,Rialto


#### FSTIP

In [21]:
df>>filter(_.mpo=='FSTIP')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
12053,Obligated,CML,5160(023),Los Banos,2014-04-16,2014-04-16,2014-04-24,2014-04-24,2014-04-24,-16940.31,...,CMAQ11-05,In City Of Los Banos,Construct/install Pedestrian/bike Improvement,4,NaT,NaT,FSTIP,5160,2014.0,Los Banos


* FSTIP is the Federal Statewide Transportation Improvement Program. 

### Number of Unique Agencies in Each MPO

In [11]:
#using _dla_utils
_dla_utils.get_nunique(df, "primary_agency_name", "mpo")

,mpo,n
18,SCAG,262
12,MTC,146
13,NON-MPO,123
26,STANCOG,90
16,SANDAG,74
22,SJCOG,73
9,MCAG,50
8,KCOG,44
14,SACOG,38
2,CFCG,24


In [14]:
#using _dla_utils
(_dla_utils.find_top((df>>filter(_.mpo=='SCAG'))))>>filter(_.variable=='primary_agency_name')

,value,count,variable
0,Los Angeles,464,primary_agency_name
1,Los Angeles County,438,primary_agency_name
2,San Bernardino County,116,primary_agency_name
3,Riverside County,112,primary_agency_name
4,Ventura County,112,primary_agency_name
5,Los Angeles County Metropolitan Transportation...,85,primary_agency_name
6,Orange County Transportation Authority,85,primary_agency_name
7,Long Beach,81,primary_agency_name
8,Santa Clarita,80,primary_agency_name
9,Santa Ana,78,primary_agency_name


## Function for Agencies in each MPO

creating a function to look into each MPO. What agencies are listed under the MPO name? Are the projects located in that jurisdiction? 

In [6]:
_dla_utils.interactive_widget_counts(df, 'mpo', 'primary_agency_name')

Dropdown(description='Mpo', options=('AMBAG', 'BCAG', 'CFCG', 'COFCG', 'ER NONE', 'FCOG', 'FSTIP', 'KCAG', 'KC…

Output()

### Filter to see values in column 

In [17]:
#using Siuba filter, we can start to look at the agenies under different MPOs

In [16]:
df>>filter(_.agency=='Marin County')>> filter(_.mpo=='SCAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
19894,Obligated,ACSTP,40A0(087),Marin County,2021-06-16,2021-06-17,2021-06-17,2021-09-13,2021-09-14,0.0,...,nan,Lucas Valley Rd At Mp 3.92,Lucas Valley Rd At Mp 3.92 Roadway Damage. Un...,2,2021-06-17,2021-06-17,SCAG,40A0,2021.0,Los Angeles County


* projet located in Lucas Valley, which is not in SCAG

In [25]:
df>>filter(_.agency=='Sonora')>> count(_.mpo)

,mpo,n
0,COFCG,1
1,NON-MPO,3
2,SCAG,2
3,STANCOG,1


In [26]:
df>>filter(_.agency=='Sonora')>> filter(_.mpo=='SCAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
8383,Obligated,CML,5010(006),Sonora,2015-07-23,2015-08-20,2015-08-26,2015-08-26,2015-08-27,0.0,...,TRANS-HUB,S. Washington St. And Stockton St/hwy 49 (see ...,"Const Ped,transit Facility And Traffic Improv",2,NaT,NaT,SCAG,5010,2015.0,Sonora
8679,Obligated,CML,5010(008),Sonora,2015-07-08,2015-07-08,2015-07-15,2015-07-17,2015-07-21,205000.0,...,SONORA-PR,244 S. Shepherd St (apn 002-164-16)/nw Corner ...,"Parking Lot, Utilize As Park & Ride (tc)",1,NaT,NaT,SCAG,5010,2015.0,Sonora


In [27]:
df>>filter(_.agency=='Shasta County')>> filter(_.mpo=='MTC')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
19491,Obligated,ACSTP,15A6(002),Shasta County,2021-05-18,2021-05-28,2021-06-02,2021-07-12,2021-07-22,0.0,...,nan,Ca20-3 Zogg Fire Eo Repairs On Platina Road,Emergency Opening,1,2021-04-15,2021-04-20,MTC,15A6,2021.0,Napa County


In [28]:
df>>filter(_.agency=='Marina')>> filter(_.mpo=='AMBAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
4849,Obligated,RPSTPL,5416(008),Marina,2017-02-28,2017-02-28,2017-02-28,2017-02-28,2017-03-10,-13679.6,...,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),7,NaT,NaT,AMBAG,5416,2017.0,Marina
5514,Obligated,RPSTPL,5416(008),Marina,2016-11-01,2016-11-01,2016-11-01,2016-11-01,2016-11-07,55689.0,...,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),6,NaT,NaT,AMBAG,5416,2016.0,Marina
5585,Obligated,RPSTPL,5416(008),Marina,2016-07-27,2016-10-25,2016-10-25,2016-10-25,2016-10-28,-55689.0,...,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),5,NaT,NaT,AMBAG,5416,2016.0,Marina
6816,Obligated,HSIPL,5416(010),Marina,2016-05-05,2016-05-05,2016-05-06,2016-05-06,2016-05-13,1300033.0,...,HSIP08,Del Monte And Beach Road Roundabout,Construct Roundabout,2,NaT,NaT,AMBAG,5416,2016.0,Marina
9170,Obligated,RPSTPL,5416(011),Marina,2015-05-21,2015-05-21,2015-05-21,2015-05-22,2015-05-27,1650000.0,...,CM01MO,Imjin Road From 4-lane Section To Reservation ...,Widen Roadway From 2 To 4 Lanes (tc),1,NaT,NaT,AMBAG,5416,2015.0,Marina
9304,Obligated,RPSTPL,5416(008),Marina,2015-05-05,2015-05-05,2015-05-05,2015-05-05,2015-05-06,2000000.0,...,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),4,NaT,NaT,AMBAG,5416,2015.0,Marina
12808,Obligated,RPSTPL,5416(008),Marina,2014-02-07,2014-02-07,2014-02-07,2014-02-07,2014-02-07,0.0,...,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),3,NaT,NaT,AMBAG,5416,2014.0,Marina
12866,Obligated,RPSTPL,5416(008),Marina,2014-01-09,2014-01-23,2014-01-28,2014-01-28,2014-01-28,0.0,...,BP01GP,Imjin Parkway From Reservation Road To Imjin Road,Construct Class Ii Bike Lanes (tc),2,NaT,NaT,AMBAG,5416,2014.0,Marina
14995,Obligated,HSIPL,5416(010),Marina,2019-10-08,2019-10-08,2019-10-08,2019-10-08,2019-10-21,-19.6,...,HSIP08,Del Monte And Beach Road Roundabout,Construct Roundabout,3,2019-10-03,2019-10-04,AMBAG,5416,2019.0,Marina


In [29]:
df>>filter(_.agency=='Kern County (District 9)')>> filter(_.mpo=='BCAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
2481,Obligated,CML,5961(006),Kern County (District 9),2018-02-05,2018-02-08,2018-02-13,2018-02-13,2018-02-21,1327950.0,...,KER160508,Backus Road From Tehachapi Willow Springs Road...,Surfacing Unpaved Shoulders,1,2018-02-02,2018-02-08,BCAG,5961,2018.0,Kern County (District 9)


In [30]:
df>>filter(_.agency=='Santa Barbara County')>> filter(_.mpo=='BCAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
293,Obligated,ER,38F0(011),Santa Barbara County,2018-09-06,2018-09-06,2018-10-29,2018-10-31,2018-11-06,83500.0,...,nan,0.35 Miles West Of The Intersection Of Skyway ...,Remove Mbgr And Install Midwest Guardrail Syst...,3,2018-07-16,2018-08-13,BCAG,38F0,2018.0,Butte County
1041,Obligated,ER,38F0(011),Santa Barbara County,2018-07-05,2018-07-05,2018-07-05,2018-07-05,2018-07-13,15000.0,...,nan,0.35 Miles West Of The Intersection Of Skyway ...,Remove Mbgr And Install Midwest Guardrail Syst...,2,2018-07-05,2018-07-05,BCAG,38F0,2018.0,Butte County


In [31]:
df>>filter(_.agency=='Bishop')>> filter(_.mpo=='KCOG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
10707,Obligated,RPSTPL,5125(021),Bishop,2014-09-23,2014-10-08,2014-10-08,2014-10-08,2014-10-14,4049.0,...,nan,Intersection Of N. 3rd Street And E. Pine Stre...,"New 1,000 Foot Paved Bike Path",5,NaT,NaT,KCOG,5125,2014.0,Bishop


In [32]:
df>>filter(_.agency=='Los Angeles')>> filter(_.mpo=='MCAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
10873,Obligated,RPSTPL,5953(685),Los Angeles,2014-09-05,2014-09-05,2014-09-08,2014-09-10,2014-09-10,378000.00,...,LAF5514,Vermont Ave - From Manchester Blvd To El Segun...,Class Ii Bike Lane Construction(3 Miles),3,NaT,NaT,MCAG,5953,2014.0,Los Angeles County
10916,Obligated,BRLSZD,5006(337),Los Angeles,2014-08-20,2014-08-20,2014-08-26,2014-08-29,2014-08-29,58000.00,...,LA000800,"Glendale-hyperion Viaduct, Bridge # 53c1882",Seismic Retrofit,8,NaT,NaT,MCAG,5006,2014.0,Los Angeles
11446,Obligated,HSIPL,5006(551),Los Angeles,2014-07-01,2014-07-01,2014-07-08,2014-07-09,2014-07-09,0.00,...,SCAG015/HSIP2-,Roxford St And I-5 (the Golden State Freeway),Install New Traffic Signal,4,NaT,NaT,MCAG,5006,2014.0,Los Angeles
11713,Obligated,CML,5006(647),Los Angeles,2014-06-10,2014-06-10,2014-06-10,2014-06-17,2014-06-18,-0.01,...,LAF1844,"City Of La, Crenshaw Blvd & Martin Luther King...",To Promote Pedestrian Activity In Project Limits,3,NaT,NaT,MCAG,5006,2014.0,Los Angeles
11754,Obligated,STPLZ,5953(564),Los Angeles,2014-04-28,2014-05-23,2014-06-02,2014-06-03,2014-06-12,767290.00,...,LA000800,East Fork Road Over The North Fork Of San Gabr...,Seismic Retrofit,4,NaT,NaT,MCAG,5953,2014.0,Los Angeles County
11873,Obligated,BRLSZD,5953(566),Los Angeles,2013-08-28,2014-05-06,2014-05-13,2014-05-20,2014-05-22,89563.00,...,LA000800,"Alameda Street Over Compton Creek, Bridge # 53...",Bridge Seismic Retrofit,3,NaT,NaT,MCAG,5953,2013.0,Los Angeles County


In [33]:
df>>filter(_.agency=='Los Angeles')>> count(_.mpo)

,mpo,n
0,ER NONE,1
1,KCOG,4
2,MCAG,6
3,NON-MPO,12
4,SANDAG,14
5,SCAG,490
6,SJCOG,4
7,STANCOG,3


In [34]:
df>>filter(_.agency=='Indio')>> filter(_.mpo=='MCTC')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
8026,Obligated,CML,5275(022),Indio,2015-10-08,2015-10-12,2015-11-10,2015-11-10,2015-11-15,81189.0,...,RIV091208,Intersection Of Jackson Street And Market Stre...,Traffic Signals And Wireless Interconnect (tc),2,NaT,NaT,MCTC,5275,2015.0,Indio


In [35]:
df>>filter(_.agency=='Los Angeles')>> filter(_.mpo=='SJCOG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
11054,Obligated,RPSTPL,5006(716),Los Angeles,2014-08-04,2014-08-04,2014-08-07,2014-08-13,2014-08-13,669000.00,...,LAF3850/LA11G2,Vermont Ave B/n Hollywood Bl & Los Angeles Cit...,"Landscape, Install 6 Fully Irrigated Medians, Etc",3,NaT,NaT,SJCOG,5006,2014.0,Los Angeles
11678,Obligated,BHLS,5006(193),Los Angeles,2014-03-28,2014-06-09,2014-06-13,2014-06-18,2014-06-19,710400.00,...,LA000800,Soto St Bridge Over Valley Blvd & Uprr; Bridge...,Bridge Rehabilitation,9,NaT,NaT,SJCOG,5006,2014.0,Los Angeles
11734,Obligated,STPL,5006(469),Los Angeles,2014-06-10,2014-06-10,2014-06-10,2014-06-12,2014-06-13,-39022.58,...,nan,"East Downtown, La (see State Comments Screen)",Truck Access Improvement,7,NaT,NaT,SJCOG,5006,2014.0,Los Angeles
11843,Obligated,HPLUL,5006(518),Los Angeles,2014-04-28,2014-05-09,2014-05-22,2014-05-23,2014-05-29,-127527.00,...,LAE2147,Devonshire St And Woodley Ave,Lighting N/w San Fernando Valley,3,NaT,NaT,SJCOG,5006,2014.0,Los Angeles


### Findings from Filtering

### MPOs with all correct agencies

* AMBAG
* KCAG
* SBCG
* TMPO

### MPOs with some misplaced agencies

* BCAG
* MTC
* SCAG
* KGOC
* MNOLTC
* FCOG 
* KCOG
* MCAG
* SJCOG
* STANCOG
* MCTC
* SACOG
* SJCOG
* STANCOG 
* TCAG


### MPOs not defined / all incorrect  
* CFCG - has Fresno Counties. Fresno is part of FCOG
* COFCG - combination of multiple MPOs
* FCOG - includes Ventura and Fresno
* FSTIP - a Prefix rather than a MPO
* MNOLTC - one entry
* SANDA - one entry
* SDAG - maybe SANDAG
* SANDAG - mostly SCAG Agencies
* SHASTA - one agency not in Shasta County
* SJCG - most likely SJCOG
* SLAC - maybe SLOCOG
* STACOG - STANCOG
* STNCOG - STANCOG
* STPA - maybe SRTA


## Agencies
Most of these agencies are corrected in the [change_agency_to_locodelist](https://github.com/cal-itp/data-analyses/blob/main/dla/e76_obligated_funds/change_agency_to_locodelist.ipynb)

In [19]:
df.agency.nunique()

671

In [18]:
df.primary_agency_name.nunique()

619

In [37]:
sorted(df.agency.unique())

['Access Services',
 'Agoura Hills',
 'Ala-Con Costa T',
 'Alameda',
 'Alameda - Contra Costa Transit District',
 'Alameda Corridor Transportation Authority',
 'Alameda County',
 'Alameda County Congestion Management Agency',
 'Alameda County Transportation Commission',
 'Alameda County Transportation Improvement Authority',
 'Alameda County Waste Management Authority',
 'Albany',
 'Alhambra',
 'Alpine County',
 'Amador',
 'Amador City',
 'Amador County',
 'Amador County Transportation Commission',
 'Amador Rapid Transit System',
 'American Canyon',
 'Anaheim',
 'Angels',
 'Antelope Valley Transit Authority',
 'Antioch',
 'Apple Valley',
 'Arcadia',
 'Arcata',
 'Arroyo Grande',
 'Artesia',
 'Arvin',
 'Association Of Monterey Bay Area Governments',
 'Atascadero',
 'Atherton',
 'Atwater',
 'Auburn',
 'Avenal',
 'Azusa',
 'Bakersfield',
 'Baldwin Park',
 'Banning',
 'Barstow',
 'Bay Area Air Quality Management District',
 'Bay Area Rt',
 'Beaumont',
 'Bell',
 'Bell Gardens',
 'Bellflower'

In [21]:
#using _dla_utils
_dla_utils.get_nunique((df>>filter(_.agency.str.contains("Amador"))), 'project_no','agency')

,agency,n
2,Amador County,11
0,Amador,4
5,Livermore Amador Valley Transit Authority,3
1,Amador City,1
3,Amador County Transportation Commission,1
4,Amador Rapid Transit System,1


* list of agencies to be corrected with incorrect locodes
    * includes the null locodes

|Project No/Correct Locode |Agency in Obligated List | Actual Name (in Locode List) | 
| --------------- | --------------- | --------------- |
| 6053 | Sanbag  | San Bernardino Associated Governments | 
| 6066 | Sd Assoc Gov'T  | San Diego Association of Governments |
| 6071 | Ora Co Trans Au  | Orange County Transportation Authority |
| 6089 | Stan Area Aog  | Stanislaus Council of Governments |
| 5181 | Huntington Beac  | Huntington Beach |
| 6054 | Riv Co Trans Co | Riverside County Transportation Commission |
| 6128 |  Calaveras C T C | Calaveras Council of Governments |
| 5417 |  La Canada-Flint | La Canada Flintridge |
| 6303 | Sgvc | San Gabriel Valley Council of Governments |
| 6349 |  Stpd | Stockton Port District |
| 6305 |  Yrts | Yosemite Area Regional Transportation System JPA |
| 6248 |  Dwr | Department of Water Resources |
| 6308 |  Jpam | Transit Joint Powers Authority For Merced County |
| 6088 |  San Joaquin Cog  | San Joaquin Council of Governments |
| 5026 |  Sanbuenaventura | San Buenaventura |
| 5372 | Sanjuancapistra | San Juan Capistrano  |
| 5478 |  Rsm | Rancho Santa Margarita |
| 6190 |  Usda Forest Ser | U.S. Forest Service, Pacific Southwest Region |
| 6498 |  Cap Soeast Con  | Capital Southeast Connector |
| 6164 | Coachella V A G |  Coachella Valley Association of Governments |
| 6061 |  Fthll/East Tca | Foothill Eastern Transportation Corridor |
| 5476 |  Lgnw |  Laguna Woods |
| 6095 | Merced Co Aog | Merced County Association of Governments |
| 5413 |  Ranchopalosverd | Rancho Palos Verdes |
| 5349 |  Rollinghillsest | Rolling Hills Estates |
| 6373 |  Samd | South Coast Air Quality Management District |
| 6245 |  Sjrt |  San Joaquin Regional Transit District |
| 5926 | Amador | Amador County |
| 6128 | Calaveras C T C | Calaveras Council of Governments |
| 5930 | Calaveras | Calaveras County |
| 6498 | Cap Soeast Con | Capital Southeast Connector |
| 6164 | Coachella | Coachella Valley Association Of Governments |
| 5384 | Dst Hot Springs | Desert Hot Springs |
| 6061 | Fthll/East Tca | Foothill Eastern Transportation Corridor |
| 5230 | Gustine City | Gustine |
| 5387 | Hawaiian Garden | Hawaiian Gardens |
| 5958 | Imperial | Imperial County |
| 4825 | Imperial County | 5958 |
| 5948 | Inyo | Inyo County |
| 5940 | Mariposa | Mariposa County |
| 40A0 | Mendocino County | 5910 |
| 5939 | Merced | Merced County |
| 0484 | Merced County | Merced County |
| 5947 | Mono | Mono County |
| 22X0 | Portola Valley | 5390 |
| 5420 | Ranchocucamonga | Rancho Cucamonga |
| 5413 | Ranchopalosverd | Rancho Palos Verdes |
| 5956 | Riverside | Riverside County |
| 5349 | Rollinghillsest | Rolling Hills Estates |
| 5954 | San Bernardino | San Bernardino  County |
| 5957 | San Diego | San Diego County |
| 5929 | San Joaquin | San Joaquin County |
| 6088 | San Joaquin Cog | San Joaquin Council Of Governments |
| 5340 | Santa Fe Sprgs | Santa Fe Springs |
| 6066 | Sd Assoc Gov’T | San Diego Association of Governments |
| 20W0 | Sierra Madre | 5158 |
| 6503 | Solano County Transit | 6497 |
| 5938 | Stanislaus | Stanislaus County |
| 5932 | Tuolumne | Tuolumne County |
| 5448 | Twentynine Palm | Twentynine Palms |
| 5952 | Ventura | Ventura County |
| 5026 | San Buena Ventura | San Buenaventura |
| 5338 | Westminister | Westminster |

### Acronym Agencies Totals

In [67]:
#https://stackoverflow.com/questions/19937362/python-pandas-filter-string-data-based-on-its-string-length
df_acronym = df[df['agency'].apply(lambda x: len(x)<5)]

In [68]:
df_acronym>>count(_.agency, _.locode)

,agency,locode,n
0,Bell,5272,12
1,Brea,5237,11
2,Brea,nan,5
3,Dwr,nan,3
4,Galt,5293,8
5,Inyo,nan,24
6,Ione,5311,2
7,Jpam,nan,3
8,Kern,5961,2
9,Lgnw,nan,1


In [69]:
df_acronym>>filter(_.agency=='Sgvc')>>count(_.project_no)

,project_no,n
0,6303(020),2
1,6303(026),1
2,6303(028),1
3,6303(030),1
4,7500(286),2


* only one acronym with a locode. checking agencies with no locode

In [72]:
# using _dla_utils
_dla_utils.get_nunique(df_acronym, 'project_no', 'agency')

,agency,n
4,Inyo,18
20,Taft,13
12,Napa,10
1,Brea,8
14,Ross,7
9,Lodi,6
13,Ojai,6
17,Sgvc,5
0,Bell,4
24,Yrts,4


* 9 agencies that have acronyms when not including locodes

In [26]:
df_acronym>>filter(_.locode.isnull())>>count(_.agency)

,agency,n


So the agency 'Sgvc' uses two 'locodes' in the `project_no` column, one of which '7500' we know is used with multiple agencies as seen in the [add_locodes](https://github.com/cal-itp/data-analyses/blob/main/dla/e76_obligated_funds/add_locodes.ipynb) notebook.

### **Agencies with Acronyms and NaN Locodes:**

In [27]:
print(len(df>>filter(_.locode.isnull())))

0


In [28]:
# number of entries with no Locode- around ~7.5%

In [29]:
print(len(df>>filter(_.locode.isnull())>>count(_.agency)))

0


In [30]:
#number of agencies with Locodes missing around ~31% 

In [31]:
df>>filter(_.locode.isnull())>>count(_.agency)>>arrange(-_.n)

,agency,n



* using the most common first four numbers of the `project_no`


|Project No/Corresponding Locode |Agency in Obligated List | Actual Name (in Locode List) | 
| --------------- | --------------- | --------------- |
| 6053 | Sanbag  | San Bernardino Associated Governments | 
| 6066 | Sd Assoc Gov'T  | San Diego Association of Governments |
| 6071 | Ora Co Trans Au  | Orange County Transportation Authority |
| 6089 | Stan Area Aog  | Stanislaus Council of Governments |
| 5181 | Huntington Beac  | Huntington Beach |
| 6054 | Riv Co Trans Co | Riverside County Transportation Commission |
| 6128 |  Calaveras C T C | Calaveras Council of Governments |
| 5417 |  La Canada-Flint | La Canada Flintridge |
| 6303 | Sgvc | San Gabriel Valley Council of Governments |
| 6349 |  Stpd | Stockton Port District |
| 6305 |  Yrts | Yosemite Area Regional Transportation System JPA |
| 6248 |  Dwr | Department of Water Resources |
| 6308 |  Jpam | Transit Joint Powers Authority For Merced County |
| 6088 |  San Joaquin Cog  | San Joaquin Council of Governments |
| 5026 |  Sanbuenaventura | San Buenaventura |
| 5372 | Sanjuancapistra | San Juan Capistrano  |
| 5478 |  Rsm | Rancho Santa Margarita |
| 6190 |  Usda Forest Ser | U.S. Forest Service, Pacific Southwest Region |
| 6498 |  Cap Soeast Con  | Capital Southeast Connector |
| 6164 | Coachella V A G |  Coachella Valley Association of Governments |
| 6061 |  Fthll/East Tca | Foothill Eastern Transportation Corridor |
| 5476 |  Lgnw |  Laguna Woods |
| 6095 | Merced Co Aog | Merced County Association of Governments |
| 5413 |  Ranchopalosverd | Rancho Palos Verdes |
| 5349 |  Rollinghillsest | Rolling Hills Estates |
| 6373 |  Samd | South Coast Air Quality Management District |
| 6245 |  Sjrt |  San Joaquin Regional Transit District |


In [32]:
#printing number of entries for Westmorland which has an offical locode of 5278 
print(len(df>>filter(_.locode=="5278")))
print(len(df>>filter(_.agency=="Westmorland")))

11
12


In [33]:
df>>filter(_.agency=="Westmorland")>>count(_.agency)

,agency,n
0,Westmorland,12


In [34]:
df>>filter(_.agency.str.contains("Huntington Beac"))>>count(_.agency)

,agency,n
0,Huntington Beac,6
1,Huntington Beach,55


#### Dwr

In [35]:
df>>filter(_.agency=="Dwr")

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
8995,Obligated,BRLOZ,6248(011),Dwr,2015-06-12,2015-06-12,2015-06-12,2015-06-12,2015-06-15,-93034.07,...,CT004,Mccabe Road Over California Aqueduct,Seismic Retrofit,2,NaT,NaT,SCAG,6248,2015.0,Department of Water Resources
8996,Obligated,BRLOZ,6248(013),Dwr,2015-06-12,2015-06-12,2015-06-12,2015-06-12,2015-06-15,-96147.13,...,CT004,Butts Road Over California Aqueduct,Seismic Retrofit,2,NaT,NaT,SCAG,6248,2015.0,Department of Water Resources
8997,Obligated,BHLOZ,6248(030),Dwr,2015-06-12,2015-06-12,2015-06-12,2015-06-12,2015-06-15,-128952.96,...,CT004,Mervel Avenue Over California Aqueduct,Seismic Retrofit,2,NaT,NaT,SCAG,6248,2015.0,Department of Water Resources


* locode list shows this as "Department of Water Resources" 

#### Jpam

In [53]:
df>>filter(_.agency=="Jpam")

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
9508,Obligated,CML,6308(016),Jpam,2015-03-23,2015-03-23,2015-04-03,2015-04-06,2015-04-07,100000.00,...,TJPA02,Merced County Region,"Public Outreach And Marketing For ""the Bus"" (tc)",1,NaT,NaT,SCAG,6308,2015.0,Transit Joint Powers Authority For Merced County
10285,Obligated,CML,6308(013),Jpam,2014-12-26,2014-12-26,2014-12-26,2014-12-26,2014-12-30,-183102.22,...,TJPA02,Merced County Transit,Accuring Bus Wraps For Public Outreach(tc),2,NaT,NaT,SCAG,6308,2014.0,Transit Joint Powers Authority For Merced County
11793,Obligated,CML,6308(014),Jpam,2014-05-20,2014-05-21,2014-06-03,2014-06-06,2014-06-06,100000.00,...,TJPA02,Merced County Region,"Public Outreach And Marketing For ""the Bus"" (tc)",1,NaT,NaT,STANCOG,6308,2014.0,Transit Joint Powers Authority For Merced County


* Showing in locode list as "Transit Joint Powers Authority For Merced County"

In [54]:
df>>filter(_.project_location=='Stanislaus County')>>count(_.agency)

,agency,n
0,Stan Area Aog,3
1,Stanislaus,2
2,Stanislaus Council Of Governments,4
3,Stanislaus County,3


In [55]:
df>>filter(_.project_location=='Stanislaus County')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
680,Obligated,CML,6089(055),Stanislaus Council Of Governments,2018-08-28,2018-08-28,2018-08-28,2018-08-29,2018-09-04,0.00,...,ST07,Stanislaus County,Regional Rideshare Includes Public Outreach,2,NaT,NaT,STANCOG,6089,2018.0,Stanislaus Council of Governments
5228,Obligated,CML,6089(062),Stanislaus Council Of Governments,2016-12-22,2016-12-22,2016-12-22,2016-12-22,2016-12-23,-4000.02,...,ST07,Stanislaus County,Regional Rideshare Program,2,NaT,NaT,STANCOG,6089,2016.0,Stanislaus Council of Governments
7372,Obligated,CML,5938(217),Stanislaus County,2016-02-17,2016-02-17,2016-02-17,2016-02-17,2016-02-23,-2915.93,...,CQSC003,Stanislaus County,Replace Motor Grader,2,NaT,NaT,STANCOG,5938,2016.0,Stanislaus County
8506,Obligated,CML,5938(232),Stanislaus,2015-08-04,2015-08-04,2015-08-12,2015-08-12,2015-08-13,71494.00,...,04STA048C,Stanislaus County,Rideshare Program,1,NaT,NaT,SCAG,5938,2015.0,Stanislaus County
9103,Obligated,CML,5938(217),Stanislaus,2015-06-02,2015-06-02,2015-06-03,2015-06-08,2015-06-09,110000.00,...,CQSC003,Stanislaus County,Replace Motor Grader,1,NaT,NaT,SCAG,5938,2015.0,Stanislaus County
10284,Obligated,STPL,6089(060),Stan Area Aog,2014-12-30,2014-12-30,2014-12-30,2014-12-30,2014-12-30,0.00,...,05STA002R,Stanislaus County,Planning And Monitoring Activities,2,NaT,NaT,SCAG,6089,2014.0,Stanislaus Council of Governments
11731,Obligated,CML,6089(062),Stan Area Aog,2014-06-04,2014-06-04,2014-06-10,2014-06-13,2014-06-16,168207.00,...,ST07,Stanislaus County,Regional Rideshare Program,1,NaT,NaT,STANCOG,6089,2014.0,Stanislaus Council of Governments
12739,Obligated,STPL,6089(056),Stan Area Aog,2014-02-18,2014-02-18,2014-02-18,2014-02-18,2014-02-20,0.00,...,05STA002R,Stanislaus County,Planning And Monitoring Activities,2,NaT,NaT,SANDAG,6089,2014.0,Stanislaus Council of Governments
14101,Obligated,STPL,6089(079),Stanislaus Council Of Governments,2019-03-11,2019-03-21,2019-03-22,2019-03-27,2019-03-28,60000.00,...,05STA002R,Stanislaus County,Planning And Monitoring Activities,1,2019-03-07,2019-03-20,STANCOG,6089,2019.0,Stanislaus Council of Governments
16633,Obligated,STPLNI,6089(079),Stanislaus Council Of Governments,2020-05-12,2020-05-12,2020-05-12,2020-05-12,2020-05-14,-0.01,...,05STA002R,Stanislaus County,Planning And Monitoring Activities,3,2020-05-07,2020-05-08,STANCOG,6089,2020.0,Stanislaus Council of Governments


* Agencies have no locodes listed under some obligations. 
* Agency name "Stanislaus" appears to be "Stanislaus County"

#### Agency Name 'YRTS'

In [56]:
df>>filter(_.agency=='Yrts')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
8716,Obligated,CML,6305(011),Yrts,2015-07-09,2015-07-13,2015-07-15,2015-07-17,2015-07-17,39283.0,...,YARTS07,"Merced, Mariposa, Mono Counties",Yarts Public Outreach And Marketing (tc),1,NaT,NaT,ER NONE,6305,2015.0,Yosemite Area Regional Transportation System JPA
10605,Obligated,CML,6305(008),Yrts,2014-10-13,2014-10-21,2014-10-23,2014-10-24,2014-10-24,45717.0,...,YARTS07,"In The Counties, Merced, Mariposa And Mono",Public Outreach And Marketing For Yarts (tc),2,NaT,NaT,SCAG,6305,2014.0,Yosemite Area Regional Transportation System JPA
11204,Obligated,CML,6305(010),Yrts,2014-07-25,2014-07-25,2014-07-29,2014-07-30,2014-07-30,85000.0,...,YARTS07,"Merced, Mariposa, Mono Counties",Yarts Public Outreach And Marketing (tc),1,NaT,NaT,SJCOG,6305,2014.0,Yosemite Area Regional Transportation System JPA
11672,Obligated,CML,6305(007),Yrts,2014-06-19,2014-06-19,2014-06-19,2014-06-19,2014-06-20,0.0,...,YARTS07,"Merced,mariposa,mono Counties",Public Outreach And Marketing,2,NaT,NaT,STANCOG,6305,2014.0,Yosemite Area Regional Transportation System JPA


In [57]:
df>>filter(_.agency=='Yrts')>>count(_.locode)

,locode,n
0,nan,4


In [58]:
df>>filter(_.agency=='Yrts')>>count(_.project_no)

,project_no,n
0,6305(007),1
1,6305(008),1
2,6305(010),1
3,6305(011),1


In [59]:
df>>filter(_.agency.str.contains('Yosemite'))>>count(_.locode)

,locode,n
0,6305,12


* 'Yrts' is actually ['Yarts'](https://yarts.com/), or Yosemite Area Regional Transportation System 

* no locodes listed, but if we use the first four numbers of the `project_no`, we get the same locode number as the `agency` Yosemite AND the official locode listed
* changing this in the "change_agency_to_locodelist" and uploading updated data in GCS

### How many agencies for a project location? 

In [38]:
#using _dla_utils
(_dla_utils.get_nunique(df, 'agency', 'project_location'))>>filter(_.n>2)

,project_location,n
8196,Various Locations,12
2623,FTA transfer,9
1830,Citywide,7
8337,Various Locations Throughout The City,6
8079,Various,5
2083,Countywide,4
7558,Stanislaus County,4
4503,Interstate 5 From S/o Avenida Pico To S/o Aven...,3
5277,Merced County,3
6925,San Francisco Bay Area,3


In [39]:
df>>filter(_.project_location.str.contains("Interstate 5 From S/o Avenida Pico To S/o Aven"))

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
4349,Obligated,STPLN,6212(016),Caltrans,2017-04-11,2017-04-11,2017-05-05,2017-05-08,2017-05-22,0.0,...,ORA990929,Interstate 5 From S/o Avenida Pico To S/o Aven...,Add Hov Lane At Each Direction (tc),4,NaT,NaT,NON-MPO,6212,2017.0,Caltrans
4656,Obligated,STPLN,6212(016),Caltrans,2016-12-05,2017-03-13,2017-03-22,2017-03-27,2017-04-07,1053000.0,...,ORA990929,Interstate 5 From S/o Avenida Pico To S/o Aven...,Add Hov Lane At Each Direction (tc),3,NaT,NaT,NON-MPO,6212,2016.0,Caltrans
11202,Obligated,CMLN,6071(071),Ora Co Trans Au,2014-07-30,2014-07-30,2014-07-30,2014-07-30,2014-07-30,0.0,...,ORA990929,Interstate 5 From S/o Avenida Pico To S/o Aven...,Add Hov Lane At Each Direction,3,NaT,NaT,MCAG,6071,2014.0,Orange County Transportation Authority
16804,Obligated,CMLN,6071(071),Orange County Transportation Authority,2020-06-25,2020-06-25,2020-06-25,2020-06-25,2020-06-26,100450000.0,...,ORA990929,Interstate 5 From S/o Avenida Pico To S/o Aven...,Add Hov Lane At Each Direction,4,2020-06-18,2020-06-19,SCAG,6071,2020.0,Orange County Transportation Authority


In [42]:
df>>filter(_.project_location.str.contains("Merced County"))>>group_by(_.agency)>>count(_.project_no, _.locode)

,agency,project_no,locode,n
0,Jpam,6308(013),nan,1
1,Jpam,6308(014),nan,1
2,Jpam,6308(016),nan,1
3,Merced,5939(094),nan,1
4,Merced County,5939(093),5939,1
5,Merced County Association Of Governments,6095(061),6095,1
6,Merced County Association Of Governments,6095(074),6095,2
7,Merced County Association Of Governments,6095(077),6095,1
8,Merced County Association Of Governments,6095(081),6095,2
9,San Joaquin Council Of Governments,6088(051),6088,3


In [65]:
#using dla_utils
_dla_utils.get_nunique((df>>filter(_.project_location.str.contains("Merced County"))), 'project_no','agency')

,agency,n
5,Transit Joint Powers Authority For Merced County,6
3,Merced County Association Of Governments,4
0,Jpam,3
4,San Joaquin Council Of Governments,2
1,Merced,1
2,Merced County,1


In [46]:
#using _dla_utils
_dla_utils.get_nunique((df>>filter(_.project_location.str.contains("San Francisco Bay Area"))), 'project_no','agency')

,agency,n
1,Metropolitan Transportation Commission,39
0,Bay Area Air Quality Management District,3
2,Metropolitan Transportation Commission - Safe,3


In [47]:
#using _dla_utils
_dla_utils.get_nunique((df>>filter(_.project_location.str.contains("San Joaquin County"))), 'project_no','agency')

,agency,n
5,San Joaquin County,6
4,San Joaquin Council Of Governments,2
0,Caltrans,1
1,Sacramento County,1
2,San Joaquin,1
3,San Joaquin Cog,1


* `agency` name 'San Joaquin' has same locode as 'San Joaquin County'

In [49]:
#using _dla_utils
_dla_utils.get_nunique((df>>filter(_.project_location=="Various Locations (see Comments)")), 'project_no','agency')

,agency,n
2,Placer County,2
0,Imperial,1
1,Imperial County,1


#### Checking whats going on in Stanislaus

In [51]:
#using _dla_utils
_dla_utils.get_nunique((df>>filter(_.project_location=="Stanislaus County")), 'project_no','agency')

,agency,n
0,Stan Area Aog,3
2,Stanislaus Council Of Governments,3
1,Stanislaus,2
3,Stanislaus County,2


* multiple `agency` names for Stanislaus County and based on the first four digits of the `project_no` we have only two locodes
    *  Stanislaus County and Stanislaus looks to be the same
   

* despite the 49 entries under Stanislaus, there are no locodes, meaning they are listed as NaN. 
* checking to see if others agencies have NaN

In [52]:
len(df>>filter(_.agency=='Stanislaus'))

49

In [53]:
df>>filter(_.agency=='Stanislaus')>>count(_.locode)

,locode,n
0,nan,49


## Non-MPO Agencies
Looking at 'Non-MPO', 'ER NONE' and 'NaN' (or null)

#### Is Null

In [54]:
df>>filter(_.mpo.isnull())

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
20446,DISTRICT,HSIPL,7504(001),Caltrans,NaT,NaT,NaT,NaT,NaT,351200.0,...,nan,FTA transfer,Guardrail Var. Locations,1,NaT,NaT,None,7504,NaN,Caltrans


### is ER NONE

In [60]:
#using _dla_utils
(_dla_utils.find_top(df>>filter(_.mpo=='ER NONE')))>>filter(_.variable=='primary_agency_name')

,value,count,variable
0,Ridgecrest,2,primary_agency_name
1,California City,1,primary_agency_name
2,Camarillo,1,primary_agency_name
3,Chino Hills,1,primary_agency_name
4,Coachella,1,primary_agency_name
5,El Centro,1,primary_agency_name
6,Los Angeles County,1,primary_agency_name
7,Mission Viejo,1,primary_agency_name
8,Palm Springs,1,primary_agency_name
9,Pico Rivera,1,primary_agency_name


In [59]:
df>>filter(_.mpo=='ER NONE')>>filter(_.agency=='Ridgecrest')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
10222,Obligated,STPL,5385(043),Ridgecrest,2015-01-02,2015-01-02,2015-01-02,2015-01-02,2015-01-05,-8826.89,...,KER050406,N. Downs Street From W. Ward Avenue To W. Inyo...,Pavement Rehabilitation,3,NaT,NaT,ER NONE,5385,2015.0,Ridgecrest
10249,Obligated,STPL,5385(044),Ridgecrest,2014-12-31,2014-12-31,2014-12-31,2014-12-31,2015-01-04,-25490.80,...,KER100405,N. Downs Street From W. Drummond Avenue To W. ...,Pavement Rehabilitation,3,NaT,NaT,ER NONE,5385,2014.0,Ridgecrest


In [61]:
#using _dla_utils
(_dla_utils.find_top(df>>filter(_.mpo=='ER NONE')))>>filter(_.variable=='dist')

,value,count,variable
0,7,5,dist
1,8,3,dist
2,9,3,dist
3,11,2,dist
4,12,2,dist
5,10,1,dist


In [62]:
df>>filter(_.mpo=='ER NONE')>>filter(_.dist==8)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
10216,Obligated,RPMSTP,5282(018),Palm Springs,2014-12-09,2014-12-22,2015-01-05,2015-01-05,2015-01-05,0.0,...,nan,Gene Autry Trail From Whitewater River Wash To...,Widen Bridge And Roadway,9,NaT,NaT,ER NONE,5282,2014.0,Palm Springs
10218,Obligated,SRTSL,5294(009),Coachella,2015-01-02,2015-01-02,2015-01-02,2015-01-02,2015-01-05,0.0,...,SCAG016,14 Locations In The Vicinity Of Five Elementar...,"Bulb-outs, Curb Extensions, Flashing Beacons, ...",5,NaT,NaT,ER NONE,5294,2015.0,Coachella
10226,Obligated,SRTSL,5467(007),Chino Hills,2014-10-30,2014-12-19,2015-01-02,2015-01-02,2015-01-05,764130.0,...,SCAG016,"Country Club Drive, Williams Avenue, Esther St...","Curb And Gutter, Sidewalk, And Ada-curb Ramps",2,NaT,NaT,ER NONE,5467,2014.0,Chino Hills


#### Is Non-MPO

In [64]:
#using _dla_utils
(_dla_utils.find_top(df>>filter(_.mpo=='NON-MPO')))>>filter(_.variable=='primary_agency_name')

,value,count,variable
0,Humboldt County,627,primary_agency_name
1,Mendocino County,239,primary_agency_name
2,Nevada County,139,primary_agency_name
3,Lake County,131,primary_agency_name
4,Calaveras County,106,primary_agency_name
5,Caltrans,105,primary_agency_name
6,Tehama County,85,primary_agency_name
7,Trinity County,80,primary_agency_name
8,Plumas County,68,primary_agency_name
9,Tuolumne County,65,primary_agency_name
